# Direct Sampling

In [1]:
import numpy as np
from analyze import morris
from sample.morris import sample


problem = {
 'num_vars': 8,
 'names': ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'],
 'groups': None,
 'bounds': [[0.0, 1.0],
            [0.0, 1.0],
            [0.0, 1.0],
            [0.0, 1.0],
            [0.0, 1.0],
            [0.0, 1.0],
            [0.0, 1.0],
            [0.0, 1.0]]
}

param_values = sample(problem, N=1000, num_levels=4, optimal_trajectories=None)

# [Define the sobol_g_function and rest of the code]
def sobol_g_function(params):
    a = [78, 12, 0.5, 2, 97, 33, 11, 90]  # Example values for 'a'
    product = 1
    for i in range(len(params)):
        product *= (abs(4 * params[i] - 2) + a[i]) / (1 + a[i])
    return product

Y = np.array([sobol_g_function(sample) for sample in param_values])

Si = morris.analyze(
    problem,
    param_values,
    Y,
    conf_level=0.95,
    print_to_console=True,
    num_levels=4,
    num_resamples=100,
)

          mu   mu_star     sigma  mu_star_conf
x1  0.000854  0.037054  0.040301      0.001027
x2 -0.007193  0.222883  0.241793      0.005866
x3  0.081879  1.595351  1.630786      0.019483
x4 -0.041108  0.873822  0.931185      0.020229
x5  0.000841  0.029838  0.032313      0.000813
x6 -0.001881  0.085133  0.092418      0.002349
x7 -0.000811  0.232832  0.253401      0.006063
x8  0.001487  0.032670  0.035447      0.000875


In [3]:
from analyze import sobol
from sample import saltelli

problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359]]
}

param_values = saltelli.sample(problem, 1024, calc_second_order=True, skip_values=2048)

def Ishigami(x, a=7, b=0.1):
    return np.sin(x[0]) + a * np.sin(x[1])**2 + b * x[2]**4 * np.sin(x[0])

Y = np.array([Ishigami(row) for row in param_values])

Si = sobol.analyze(
    problem, Y, calc_second_order=True, conf_level=0.95, print_to_console=True
)

          ST   ST_conf
x1  0.557947  0.082088
x2  0.442189  0.038065
x3  0.241402  0.026503
          S1   S1_conf
x1  0.310576  0.059117
x2  0.443653  0.051667
x3 -0.012962  0.065608
                S2   S2_conf
(x1, x2) -0.014397  0.087689
(x1, x3)  0.246231  0.113027
(x2, x3)  0.000539  0.060319


C:\Users\sharm\AppData\Local\Temp\ipykernel_13756\723000862.py:12: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 1024, calc_second_order=True, skip_values=2048)
